# Tune angle for GIWAXS
### First steps
Run "make_SAXS_macro.py" or the following cell

In [1]:
import make_SAXS_macro as mk_macro

start = 0.0     # angle to start sweep at
finish = 1.2    # last angle to include
step = 0.005    # how much angle to change between each exposure
tag = ""        # optional text to add to file name

# create list of angles to scan
angles = mk_macro.arange(start, finish, step)

# create file in Macros directory
mk_macro.create_file(angles, tag)

[0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08, 0.085, 0.09, 0.095, 0.1, 0.105, 0.11, 0.115, 0.12, 0.125, 0.13, 0.135, 0.14, 0.145, 0.15, 0.155, 0.16, 0.165, 0.17, 0.175, 0.18, 0.185, 0.19, 0.195, 0.2, 0.205, 0.21, 0.215, 0.22, 0.225, 0.23, 0.235, 0.24, 0.245, 0.25, 0.255, 0.26, 0.265, 0.27, 0.275, 0.28, 0.285, 0.29, 0.295, 0.3, 0.305, 0.31, 0.315, 0.32, 0.325, 0.33, 0.335, 0.34, 0.345, 0.35, 0.355, 0.36, 0.365, 0.37, 0.375, 0.38, 0.385, 0.39, 0.395, 0.4, 0.405, 0.41, 0.415, 0.42, 0.425, 0.43, 0.435, 0.44, 0.445, 0.45, 0.455, 0.46, 0.465, 0.47, 0.475, 0.48, 0.485, 0.49, 0.495, 0.5, 0.505, 0.51, 0.515, 0.52, 0.525, 0.53, 0.535, 0.54, 0.545, 0.55, 0.555, 0.56, 0.565, 0.57, 0.575, 0.58, 0.585, 0.59, 0.595, 0.6, 0.605, 0.61, 0.615, 0.62, 0.625, 0.63, 0.635, 0.64, 0.645, 0.65, 0.655, 0.66, 0.665, 0.67, 0.675, 0.68, 0.685, 0.69, 0.695, 0.7, 0.705, 0.71, 0.715, 0.72, 0.725, 0.73, 0.735, 0.74, 0.745, 0.75, 0.755, 0.76, 0.765, 0.77, 

## Load data set

In [1]:
import numpy as np
import matplotlib.pylab as plt
import plot_2d
%matplotlib qt
import loader
import show_tiff
import fitting
import tkinter

default_path = "/home/saxs/DATA"

# Create Tk root
root = tkinter.Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

# get list of paths (strings) to each data file
# tifs = loader.search_files(default_path)
# tifs = loader.get_everthing(
#         "C:\\Users\\Teddy\\OneDrive - UCB-O365\\Rogerslab3\\Teddy\\XRD\\Silicon-silica-TT5-GIWAXS tune 2023-10-10")
tifs = loader.get_everthing(
         "C:\\Users\\Teddy\\OneDrive - UCB-O365\\Rogerslab3\\Teddy\\TPP Films\\Unkown inclusion (benz, tolu, xyl)\\giwaxs\\tune")
"""
Load data as numpy arrays.
angles = 1D array of motor angle settings
intenisty_data = 3D array of intesnities; indices are (angle_index, z_index, x_index)
direct_beam = 2D array of intensity of the direct beam exposure; indices are (z_index, x_index)
"""
angles, intensity_data, direct_beam = loader.load_files(tifs)

print("angles shape: {}".format(angles.shape))
print("intensity_data shape: {}".format(intensity_data.shape))
print("direct_beam shape: {}".format(direct_beam.shape))

%gui tk

C:Users\Teddy\OneDrive - UCB-O365\Rogerslab3\Teddy\TPP Films\Unkown inclusion (benz, tolu, xyl)\giwaxs\tune


ERROR:root:Cannot activate multiple GUI eventloops


angles shape: (240,)
intensity_data shape: (240, 1065, 1030)
direct_beam shape: (1065, 1030)


Optional: Display a file

In [2]:
clip = None     # clip intensity

show_tiff.show(direct_beam, clip)
plt.show()

### Crop data
Will animate data to confirm the crop

In [4]:
x_pixel_width = 13  # number of pixels to sum across horizontally centered around the beam center
pixels_above = 50   # number of pixels above the beam to keep (for crop)
pixels_below = 20   # number of pixels below the beam to keep (for crop)
pixel_size = 0.075  # mm/pixel

"""Crop data"""
id_c, db_c = loader.crop_data(intensity_data, direct_beam, x_pixel_width, pixels_above, pixels_below)

"""Animate"""
frame_delay = 10       # milliseconds
clip = 20

fig, ani = show_tiff.animate(id_c, frame_delay, clip)

"""Plot 2D"""
id_c = np.sum(id_c, axis=2)
db_c = np.sum(db_c, axis=1)
z_px = np.arange(len(db_c))
z = (z_px[::-1] + 0.5) * pixel_size

print(angles.shape)
print(z.shape)
print(id_c.shape)
plot_2d.plot_all(angles, z, id_c)

plt.show()

(240,)
(70,)
(240, 70)


### Find direct beam with gaussian fit

In [18]:
# db_gfit = (amplitude (counts), center (z-pixels), sigma (z-pixels)) 
db_gfit = fitting.gaussian_constrained_fit(z, db_c)
print(f"amplitude = {gfit[0]} counts")
print(f"center = {gfit[1]} pixels")
print(f"")
#db_first_pixel = np.where(gaussian0(z, *gfit) > 1)[0][0]

[-8.31274458e+04  5.97319976e+01 -1.18291069e+03]
